# Imports

In [ ]:
# Import mesa classes
from mesa import Agent, Model
from mesa.datacollection import DataCollector
from mesa.time import StagedActivation
from mesa.space import MultiGrid
from mesa.batchrunner import BatchRunner

# Import other libraries
import numpy as np
import scipy.stats as stats
import random
from operator import attrgetter
from collections import defaultdict

# Global constants

In [ ]:
class Constants(object):
    GRID_SIZE = 20
    TICK_DURATION = 0.2 # 1 day == 5 actions
    CANVAS_SIZE = 400
    
    # ENVIRONMENT
    TOURIST_SPOT_RATIO = 0.2
    
    # AGENTS
    AGENT_CHOOSE_DESTINATION_PROBABILITY = 0.5
    PRO_TOURISM_DISSATISFACTION_WEIGHT = 0.8 # reduces dissatisfaction of locals who benefit from tourism 
    
    # SUBMODELS
    
    # 1. POLLUTION
    POLLUTION_RATE = 0.1
    POLLUTION_CLEAN_RATE = 0.01
    LOCAL_POLLUTE_PROBABILITY = 0.2
    
    # 2. OVERCROWDING
    MAX_OVERCROWDING_THRESHOLD = 15 # number of agents in neighbourhood that causes maximum overcrowding
    
    # 3. OVERCAPACITY
    MAX_EXCESS_CAPACITY_THRESHOLD = 4 # number of agents above the toursit spot capacity that causes maximum overcapcity
    TOURIST_SPOT_CAPACITY_MEAN = 4
    TOURIST_SPOT_CAPACITY_STD = 0
    TOURIST_SPOT_CAPACITY_MIN = 4
    
    # 4. PLEASURE
    TOURIST_SPOT_APPEAL_ALPHA = 3.5
    TOURIST_SPOT_APPEAL_MIN = 0.4
    TOURIST_SPOT_APPEAL_MAX = 5
    TOURIST_SPOT_PRICE_MEAN = 1
    TOURIST_SPOT_PRICE_STD = 0
    NEGATIVE_PLEASURE_WEIGHT = 1
    
    # 5. ECONOMIC
    LOCAL_INCOME_MEAN = 30
    LOCAL_INCOME_STD = 10
    LOCAL_WEALTH_MEAN = 1000
    LOCAL_WEALTH_STD = 0
    N_INCOME_HISTORY = 10
    
    # TOURIST STAY DURATION
    TOURIST_STAY_DURATION_MEAN = 7
    TOURIST_STAY_DURATION_STD = 3
    TOURIST_UTILITY_LOW = -1
    TOURIST_UTILITY_HIGH = 1

In [ ]:
class Helper(object):
    def get_squared_torus_distance(pos_1, pos_2, grid_size):
        # Get the squared torus distance between two points
        x1, y1 = pos_1
        x2, y2 = pos_2
        dx = abs(x1 - x2)
        dy = abs(y1 - y2)
        return min(dx, grid_size - dx) ** 2 + min(dy, grid_size - dy) ** 2
    
    def get_utility_colors_local():
        colors = []
        # Code snippet below is a modified version of the code in https://stackoverflow.com/questions/4161369/html-color-codes-red-to-yellow-to-green
        red = 255
        green = 0
        blue = 0
        step_size = 255 * 2 // 11
        
        while green < 255:
            green = min(green + step_size, 255)
            colors.append('#%02x%02x%02x' % (red, green, blue))
        
        while red > 0:
            red = max(red - step_size, 0)
            colors.append('#%02x%02x%02x' % (red, green, blue))
        return colors
    
    def get_utility_colors_tourist():
        colors = []
        # Code snippet below is a modified version of the code in https://stackoverflow.com/questions/4161369/html-color-codes-red-to-yellow-to-green
        red = 0
        green = 0
        blue = 255
        step_size = 255 * 2 // 11
        
        while green < 255:
            green = min(green + step_size, 255)
            colors.append('#%02x%02x%02x' % (red, green, blue))
        
        while blue > 0:
            blue = max(blue - step_size, 0)
            colors.append('#%02x%02x%02x' % (red, green, blue))
        return colors
    
    UTILITY_COLORS_LOCAL = get_utility_colors_local()
    UTILITY_COLORS_TOURIST = get_utility_colors_tourist()
    
    def get_utility_color(utility, agent):
        normalised_utility = int((2.5 + min(max(utility, -2.5), 2.5)) / 0.5) # cap utility
        return Helper.UTILITY_COLORS_LOCAL[normalised_utility] if agent.is_local else Helper.UTILITY_COLORS_TOURIST[normalised_utility]
    
    flatten = lambda t: [item for sublist in t for item in sublist]
    
    MODEL_REPORTERS = {
        'mean_local_utility' : lambda m : np.mean([a.utility for a in m.local_agents]) if m.local_agents else 0,
        'mean_local_pt_utility' : lambda m : np.mean([a.utility for a in m.local_agents if a.is_pro_tourism]) if any([a.utility for a in m.local_agents if a.is_pro_tourism]) else 0,
        'mean_local_npt_utility' : lambda m : np.mean([a.utility for a in m.local_agents if not a.is_pro_tourism]) if any([a.utility for a in m.local_agents if not a.is_pro_tourism]) else 0,
        'mean_tourist_utility' : lambda m : np.mean([a.utility for a in m.tourist_agents]) if m.tourist_agents else 0,
        'mean_tourist_utility_hist' : lambda m : np.mean(Helper.flatten(m.mean_tourist_utility_hist)) if Helper.flatten(m.mean_tourist_utility_hist) else 0,
        'mean_pollution' : lambda m : np.mean([a.pollution for a in m.street_patches]),
        'n_tourists' : lambda m : len(m.tourist_agents),
        'town_wealth' : lambda m : m.town_wealth_hist,
        # LOCALS
        'mean_overcapacity_lpt' : lambda m : m.get_mean_factor_for_locals('overcapacity', True),
        'mean_overcrowding_lpt' : lambda m : m.get_mean_factor_for_locals('overcrowding', True),
        'mean_pollution_lpt' : lambda m : m.get_mean_factor_for_locals('pollution', True),
        'mean_pleasure_lpt' : lambda m : m.get_mean_factor_for_locals('pleasure', True),
        'mean_wealth_change_lpt' : lambda m : m.get_mean_factor_for_locals('wealth_change', True),
        'mean_overcapacity_lnpt' : lambda m : m.get_mean_factor_for_locals('overcapacity', False),
        'mean_overcrowding_lnpt' : lambda m : m.get_mean_factor_for_locals('overcrowding', False),
        'mean_pollution_lnpt' : lambda m : m.get_mean_factor_for_locals('pollution', False),
        'mean_pleasure_lnpt' : lambda m : m.get_mean_factor_for_locals('pleasure', False),
        'mean_wealth_change_lnpt' : lambda m : m.get_mean_factor_for_locals('wealth_change', False),
        # TOURISTS
        'mean_overcapacity_t' : lambda m : np.mean([a.curr_utilities['overcapacity'] for a in m.tourist_agents]) if m.tourist_agents else 0,
        'mean_overcrowding_t' : lambda m : np.mean([a.curr_utilities['overcrowding'] for a in m.tourist_agents]) if m.tourist_agents else 0,
        'mean_pollution_t' : lambda m : np.mean([a.curr_utilities['pollution'] for a in m.tourist_agents]) if m.tourist_agents else 0,
        'mean_pleasure_t' : lambda m : np.mean([a.curr_utilities['pleasure'] for a in m.tourist_agents]) if m.tourist_agents else 0,
        'mean_mood_t' : lambda m : np.mean([a.curr_utilities['mood'] for a in m.tourist_agents]) if m.tourist_agents else 0,
        # DEBUG
        'debug' : lambda m : m.debug
    }
    
    AGENT_REPORTERS = {
        'is_local' : lambda a: a.is_local,
        'utility' : lambda a: a.utility,
        'is_pro_tourism' : lambda a: a.is_pro_tourism if a.is_local else False,
        'overcapacity' : lambda a: a.curr_utilities['overcapacity'],
        'overcrowding' : lambda a: a.curr_utilities['overcrowding'],
        'pollution' : lambda a: a.curr_utilities['pollution'],
        'mood' : lambda a: a.curr_utilities['mood'],
    }

# Agent

In [ ]:
class MyAgent(Agent):
    """Base agent class."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        # State variables
    
    """
    STAGED ACTIVATION
    """
    def pre_action(self):
        pass
    
    def action(self):
        pass
    
    def post_action(self):
        pass
    
    """
    VISUALISATION
    """
    def get_portrayal(self):
        return {}
    
    """
    HELPER METHODS
    """
    def get_neighbor_agents(self, agent_classes=[], include_center=True, radius=1, pos=None):
        if pos is None:
            pos = self.pos
        neighbors = self.model.grid.get_neighbors(pos=pos, moore=True, include_center=include_center, radius=radius)
        if self in neighbors:
            neighbors.remove(self)
        neighbors = [n for n in neighbors if any([issubclass(type(n), c) for c in agent_classes])]
        
        return neighbors
    
    def get_patch(self):
        return self.get_neighbor_agents(agent_classes=[MyPatch], radius=0)[0]

In [ ]:
class GridAgent(MyAgent):
    """Base class for Tourist and Local agents."""
    
    def __init__(self, unique_id, model, polluting_probability=0.1):
        super().__init__(unique_id, model)
        self.curr_utilities = defaultdict(int) # stores the changes in utilities for this time step
        self.utility = 0 # utility score
        self.curr_day_utilities = []
        self.chosen_destination = None
        self.polluting_probability = polluting_probability
        r = lambda: self.random.randint(0,255)
        self.color = '#%02X%02X%02X' % (r(),r(),r())
        self.is_local = False
        self.movement_behaviour = ''
        self.random_walk_dx = 0
        self.random_walk_dy = 0
        self.pleasure_hist = { i : i.pleasure_score_for_agent(self) for i in model.tourist_spot_patches}
        self.visited_location = None
        self.initial_pos = None
        self.explore_ticks = 0
        self.tick = 0
    
    """
    STAGED ACTIVATION
    """
    def pre_action(self):
        self.curr_utilities = defaultdict(int) # reset current utilities
        self.visited_location = None # reset visited location
        self.tick += Constants.TICK_DURATION
    
    """
    ACTION
    """
    def move(self):
        patch = self.get_patch()
        
        # Ensure the agent has a movement behaviour selected
        if self.movement_behaviour == '':
            self.chosen_destination = self.choose_destination()
            self.movement_behaviour = 'to_tourist_spot'
            
            # if agent has not chosen a tourist spot, set movement behaviour to explore
            if not self.chosen_destination:
                self.movement_behaviour = 'explore'
                self.explore_ticks = 3
                self.random_walk_dx = np.random.randint(-1, 2)
                self.random_walk_dy = np.random.randint(-1, 2)
        
        # If agent is currently moving to a tourist destination
        if self.movement_behaviour == 'to_tourist_spot':
            # move agent towards chosen destination
            self.move_towards(self.chosen_destination.pos)
            # if agent has arrived at destination, attempt to visit it
            if self.chosen_destination == patch:
                self.visit_patch(patch)
                self.movement_behaviour = 'return_home'
        # else if agent is currently exploring
        elif self.movement_behaviour == 'explore':
            # walk in a random direction for n-time steps
            self.move_randomly()
            self.explore_ticks -= 1
            
            if self.explore_ticks == 0:
                self.movement_behaviour = ''
        # else if agent is returning home
        elif self.movement_behaviour == 'return_home':
            self.move_towards(self.initial_pos)
            if patch.pos == self.initial_pos:
                self.movement_behaviour = ''
    
    def choose_destination(self):
        """
        Choose a "nearby" tourist spot to visit.
        Next destination is chosen based on historical pleasure score such that tourist spots with
        higher historical pleasure score have higher probability of being chosen.
        """
        destination = None
        
        # Agents choose destinations that are close to where they are right now.
        nearby_tourist_spots = self.get_neighbor_agents(pos=self.pos, agent_classes=[TouristSpot], include_center=True, radius=3)
        # only consider tourist spots that is perceived to give positive pleasure utility
        nearby_tourist_spots = [i for i in nearby_tourist_spots if self.pleasure_hist[i] > 0]

        if nearby_tourist_spots:
            weights = [self.pleasure_hist[i] for i in nearby_tourist_spots]
            destination = self.random.choices(nearby_tourist_spots, weights=weights)[0]
        
        return destination
    
    def move_towards(self, pos):
        """
        Step one cell towards destination.
        """
        next_moves = self.model.grid.get_neighborhood(self.pos, True, True)
        next_move = min(next_moves, key=lambda x : Helper.get_squared_torus_distance(x, pos, Constants.GRID_SIZE))
        self.model.grid.move_agent(self, next_move)
    
    def move_randomly(self):
        """
        Step one cell towards a random direction (i.e. explore the place).
        """
        next_move = (self.pos[0] + self.random_walk_dx, self.pos[1] + self.random_walk_dy)
        self.model.grid.move_agent(self, next_move)
    
    def visit_patch(self, patch):
        """
        Agents can visit tourist spots given there is sufficient capacity.
        """
        retval = 0 # Default value
        if patch == self.chosen_destination:
            pleasure = 0
            overcapacity = 0
            n_occupants = patch.get_n_occupants()
            self.visited_location = patch
            
            if n_occupants <= patch.capacity:
                # Agent successfully visited patch, increase utility from pleasure
                pleasure = 1
                retval = 1
            else:
                # Agent was unable to visit location due to overcapacity
                overcapacity = -min((n_occupants - patch.capacity) / Constants.MAX_EXCESS_CAPACITY_THRESHOLD, 1.0)
                retval = -1 # Not enough capacity
            
            self.update_curr_utility('overcapacity', overcapacity)
            self.update_curr_utility('pleasure', pleasure)
            
            self.chosen_destination = None # reset destination
        
        return retval
    
    def maybe_pollute(self, patch):
        """
        With a certain probability add pollution to the patch.
        """
        if self.random.random() < self.polluting_probability:
            patch.pollution = min(patch.pollution + Constants.POLLUTION_RATE, 1.0)
    
    """
    SENSE
    """
    def sense_pollution(self):
        """
        Sense pollution in tile. Adjust utility accordingly.
        """
        # Decrease utility due to pollution
        pollution_utility = -np.mean([p.pollution for p in self.get_neighbor_agents([MyPatch])])
        self.update_curr_utility('pollution', pollution_utility)
    
    def sense_overcrowding(self):
        """
        Sense overcrowding in neighbouring tiles. Adjust utility accordingly.
        """
        # Decrease utility due to overcrowding
        n_other_agents = len(self.get_neighbor_agents([GridAgent])) - 1 # exclude self
        overcrowding = -min(n_other_agents / Constants.MAX_OVERCROWDING_THRESHOLD, 1)
        self.update_curr_utility('overcrowding', overcrowding)
    
    def sense_pleasure(self):
        curr_patch = self.get_patch()
        
        for tourist_spot in self.model.tourist_spot_patches:
            initial_pleasure = tourist_spot.pleasure_score_for_agent(self)
            curr_pleasure = self.pleasure_hist[tourist_spot]
            new_pleasure = min(curr_pleasure + 0.01 * initial_pleasure, initial_pleasure) # by default, replenish pleasure score by 1%
            
            # If agent visited this place, evaluate its pleasure score
            if self.curr_utilities['pleasure'] > 0 and tourist_spot == self.visited_location:
                factors = ['overcrowding', 'pollution', 'overcapacity']
                alpha = Constants.NEGATIVE_PLEASURE_WEIGHT * np.mean([(self.curr_utilities[i] / self.model.utility_weights[i]) if self.model.utility_weights[i] else 0 for i in factors])
                new_pleasure = max(min(curr_pleasure + np.round(alpha * initial_pleasure), initial_pleasure), 0)
                self.curr_utilities['pleasure'] = new_pleasure
            
            self.pleasure_hist[tourist_spot] = new_pleasure
    
    """
    UTILITY
    """
    def update_curr_utility(self, utility, value):
        self.curr_utilities[utility] = self.model.utility_weights[utility] * value
    
    """
    HELPER
    """
    def pleasure_score(self, tourist_spot):
        return self.pleasure_hist[tourist_spot]

In [ ]:
class LocalAgent(GridAgent):
    """Local agent class."""
    def __init__(self, unique_id, model, polluting_probability=0, is_pro_tourism=False, income=0, wealth=0):
        GridAgent.__init__(self, unique_id, model, polluting_probability)
        self.is_local = True
        self.is_pro_tourism = is_pro_tourism
        self.base_income = income
        self.income = Constants.N_INCOME_HISTORY * [income] #keep track of income history
        self.wealth = Constants.N_INCOME_HISTORY * [wealth] #keep track of wealth history
    
    """
    VISUALISATION
    """
    def get_portrayal(self):
        mode = self.model.agent_portrayal
        color = self.color
        if mode == 'utility':
            color = Helper.get_utility_color(self.utility, self)
        
        return {
            'Shape' : 'rect',
            'w' : 0.5,
            'h' : 0.5,
            'Color' : color,
            'Filled' : 'true',
            'Layer' : 1
        }
    
    """
    STAGED ACTIVATION
    """
    def action(self):
        patch = self.get_patch()
        
        self.move()
        self.maybe_pollute(patch)
    
    def post_action(self):
        self.receive_income()
        self.update_income_and_wealth()
        self.update_utility()
        
    
    """
    ACTION
    """
    def receive_income(self):
        """
        Gain wealth from income
        """
        self.wealth[-1] += self.income[-1]
    
    def visit_patch(self, patch):
        retval = super().visit_patch(patch)
        
        if retval > 0: # if successfully visit tourism spot
            self.wealth[-1] -= patch.price_for_agent(self) # reduce current wealth by the price of the attraction
        
        return retval
    
    """
    SENSE
    """
    def sense_wealth_change(self):
        # Increase/Decrease wealth if it changes
        mean_wealth = np.mean(self.wealth)
        wealth_change = (self.wealth[-1] - mean_wealth) / mean_wealth
        self.update_curr_utility('wealth_change', wealth_change)
    
    """
    UTILITY
    """
    def update_utility(self):
        self.sense_overcrowding()
        self.sense_pollution()
        self.sense_wealth_change()
        self.sense_pleasure() # should be called last after all other utilities
        
        # add the current tick utilities to the current day's utility score.
        self.curr_day_utilities.append(sum([v if v > 0 else v * Constants.PRO_TOURISM_DISSATISFACTION_WEIGHT for k, v in self.curr_utilities.items()]))
        
        #if self.tick.is_integer(): # update utility once per day.
        self.utility = np.mean(self.curr_day_utilities)
        self.curr_day_utilities = []
    
    """
    INCOME
    """
    def update_income_and_wealth(self):
        # Delete oldest income/wealth
        self.income.pop(0)
        self.wealth.pop(0)
        # Copy latest income/wealth
        self.income.append(self.income[-1])
        self.wealth.append(self.wealth[-1])

In [ ]:
class TouristAgent(GridAgent):
    """Tourist agent class."""
    def __init__(self, unique_id, model, polluting_probability=0, stay_duration=7):
        GridAgent.__init__(self, unique_id, model, polluting_probability)
        # State variables
        self.is_local = False
        self.stay_duration = stay_duration
    
    """
    VISUALISATION
    """
    def get_portrayal(self):
        mode = self.model.agent_portrayal
        color = self.color
        if mode == 'utility':
            color = Helper.get_utility_color(self.utility, self)
        
        return {
            'Shape' : 'circle',
            'r' : 0.5,
            'Color' : color,
            'Filled' : 'true',
            'Layer' : 1
        }
    
    """
    STAGED ACTIVATION
    """
    def action(self):
        patch = self.get_patch()
        self.move()
        self.maybe_pollute(patch)
    
    def post_action(self):
        self.update_utility()
        self.stay_or_leave()
    
    """
    ACTION
    """
    def visit_patch(self, patch):
        retval = super().visit_patch(patch)
        
        if retval > 0: # if successfully visit tourism spot
            price = patch.price_for_agent(self)
            self.model.town_wealth += price # increment town wealth
        
        return retval
    
    def stay_or_leave(self):
        if self.utility < Constants.TOURIST_UTILITY_LOW or self.tick >= self.stay_duration:
            self.model.mean_tourist_utility_hist[-1].append(self.utility) # record how satisfied the agent was during their stay
            self.model.remove_agent(self)
            
            # To simulate exponential growth in tourism, each agent who leaves
            # has a probability of introducing new tourists.
            if self.random.random() < 0.5:
                if self.utility > Constants.TOURIST_UTILITY_HIGH:
                    self.model.n_new_tourists = 2
                elif self.utility > Constants.TOURIST_UTILITY_LOW:
                    self.model.n_new_tourists = 1
    
    """
    UTILITY
    """
    def update_utility(self):
        self.sense_overcrowding()
        self.sense_pollution()
        self.sense_mood()
        self.sense_pleasure() # should be called last after all others
        
        # add the current tick utilities to the current day's utility score.
        self.curr_day_utilities.append(sum([v for k, v in self.curr_utilities.items()]))
        
        if self.tick.is_integer(): # update utility once per day.
            self.utility = np.mean(self.curr_day_utilities)
            self.curr_day_utilities = []
    
    """
    SENSE
    """
    def sense_mood(self):
        # Sense mood of locals nearby. If people are satisfied, increase utiltiy, vice versa
        neighbors = self.get_neighbor_agents([LocalAgent])
        mean_mood = np.mean([a.utility for a in neighbors]) if neighbors else 0.0
        self.update_curr_utility('mood', mean_mood)

# Patch

In [ ]:
class MyPatch(MyAgent):
    """Base patch class."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        # State variables
        self.pollution = 0 # pollution level
        self.is_tourist_spot = type(self) is TouristSpot
    
    """
    VISUALISATION
    """
    def get_portrayal(self):
        return {
            'Shape' : 'rect',
            'w' : 1,
            'h' : 1,
            'Color' : 'White',
            'Filled' : 'true',
            'Layer' : 0
        }
    
    """
    HELPER
    """
    def get_n_occupants(self):
        return len(self.get_neighbor_agents(agent_classes=[TouristAgent, LocalAgent], radius=0))

In [ ]:
class TouristSpot(MyPatch):
    """Tourst spot patch class."""
    def __init__(self, unique_id, model, capacity=0):
        super().__init__(unique_id, model)
        # State variables
        self.capacity = capacity
        self.appeal = 0
        self.price = 0
    
    def initialise(self, appeal, price):
        self.appeal = appeal
        self.price = price
    
    """
    VISUALISATION
    """
    def get_portrayal(self):
        portrayal = super().get_portrayal()
        portrayal['Color'] = 'Grey'
        return portrayal
    
    """
    HELPER
    """
    def pleasure_score_for_agent(self, agent):
        return self.appeal - self.price
        
    def price_for_agent(self, agent):
        """
        Returns the price of visiting this tourist spot based on the agent who visits
        """
        price = self.price
        
        if not agent.is_local and self.model.policy == 'tax':
            price *= self.model.tax_rate
        
        return price

In [ ]:
class Street(MyPatch):
    """Street patch class."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        # State variables
    
    """
    VISUALISATION
    """
    def get_portrayal(self):
        portrayal = super().get_portrayal()
        portrayal['Color'] = '#%02x%02x%02x' % (255, min(int(255 * (1 - self.pollution)), 255), 255)
        return portrayal

# Model

In [ ]:
class OvertourismModel(Model):
    """Overtourism model"""
    def __init__(self,
                 n_locals,
                 n_initial_tourists,
                 grow_n_tourists,
                 pro_tourism_probability,
                 agent_portrayal,
                 collect_data,
                 policy,
                 tax_rate,
                 travel_limit,
                 # utility weights
                 utility_weight_pleasure,
                 utility_weight_overcrowding,
                 utility_weight_overcapacity,
                 utility_weight_pollution,
                 utility_weight_wealth_change,
                 utility_weight_mood,
                 # end: utility weights
                 grid_size = Constants.GRID_SIZE
                ):
        # Model variables
        self.current_id = 0
        self.n_locals = n_locals
        self.n_initial_tourists = n_initial_tourists
        self.grow_n_tourists = grow_n_tourists
        self.grid_size = grid_size
        self.tourist_spot_patches = set()
        self.street_patches = set()
        self.local_agents = set()
        self.tourist_agents = set()
        self.town_wealth = 0
        self.agent_portrayal = agent_portrayal
        self.pro_tourism_probability = pro_tourism_probability
        self.collect_data = collect_data
        self.utility_weights = {
            'pleasure' : utility_weight_pleasure,
            'overcrowding' : utility_weight_overcrowding,
            'overcapacity' : utility_weight_overcapacity,
            'pollution' : utility_weight_pollution,
            'wealth_change' : utility_weight_wealth_change,
            'mood' : utility_weight_mood,
        }
        self.mean_tourist_utility_hist = [[] for i in range(20)]
        self.town_wealth_hist = 0
        self.n_new_tourists = 0
        self.debug = ''
        
        # Policy related
        self.policy = policy
        self.tax_rate = tax_rate
        self.travel_limit = travel_limit
        
        # Model components
        self.grid = MultiGrid(self.grid_size, self.grid_size, True)
        self.schedule = StagedActivation(self, ['pre_action', 'action', 'post_action'], shuffle=True)
        self.running = True # required for batch runs
        self.datacollector = DataCollector(model_reporters=Helper.MODEL_REPORTERS, agent_reporters=Helper.AGENT_REPORTERS)
        
        # Create patches
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                p = None
                
                if self.random.random() < Constants.TOURIST_SPOT_RATIO:
                    p = TouristSpot(self.next_id(),
                                    self,
                                    capacity=max(round(np.random.normal(Constants.TOURIST_SPOT_CAPACITY_MEAN, Constants.TOURIST_SPOT_CAPACITY_STD)), Constants.TOURIST_SPOT_CAPACITY_MIN),
                                   )
                    self.tourist_spot_patches.add(p)
                else:
                    p = Street(self.next_id(), self)
                    self.street_patches.add(p)
                self.grid.place_agent(p, (x, y))
        
        # Initialise appeal
        tourist_spot_patches_list = list(self.tourist_spot_patches)
        appeals = np.random.power(Constants.TOURIST_SPOT_APPEAL_ALPHA, len(tourist_spot_patches_list))
        appeal_range = Constants.TOURIST_SPOT_APPEAL_MAX - Constants.TOURIST_SPOT_APPEAL_MIN
        for i in range(len(self.tourist_spot_patches)):
            appeal = 1#Constants.TOURIST_SPOT_APPEAL_MIN + np.ceil(appeal_range * (1.0 - appeals[i]))
            price = 0
            #price=round(np.random.normal(Constants.TOURIST_SPOT_PRICE_MEAN, Constants.TOURIST_SPOT_PRICE_STD))       
            tourist_spot_patches_list[i].initialise(appeal=appeal, price=price)
            
        # Create local agents
        for i in range(self.n_locals):
            a = LocalAgent(self.next_id(),
                           self,
                           polluting_probability=Constants.LOCAL_POLLUTE_PROBABILITY,
                           is_pro_tourism=self.random.random() < self.pro_tourism_probability,
                           income=round(np.random.normal(Constants.LOCAL_INCOME_MEAN, Constants.LOCAL_INCOME_STD)),
                           wealth=round(np.random.normal(Constants.LOCAL_WEALTH_MEAN, Constants.LOCAL_WEALTH_STD)),
                          )
            self.add_agent(a, self.get_random_coordinate())
        
        # Create tourist agents
        for i in range(self.n_initial_tourists):
            self.add_new_tourist_agent()
    
    def get_random_coordinate(self):
        return (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
    
    def add_new_tourist_agent(self):
        a = TouristAgent(self.next_id(),
                         self,
                         polluting_probability=Constants.LOCAL_POLLUTE_PROBABILITY,
                         stay_duration=max(round(np.random.normal(Constants.TOURIST_STAY_DURATION_MEAN, Constants.TOURIST_STAY_DURATION_STD)), 1),
                        )
        self.add_agent(a, self.get_random_coordinate())
    
    def add_agent(self, agent, pos):
        self.schedule.add(agent)
        self.grid.place_agent(agent, pos=pos)
        agent.initial_pos = pos
        
        if type(agent) is LocalAgent:
            self.local_agents.add(agent)
        elif type(agent) is TouristAgent:
            self.tourist_agents.add(agent)
    
    def remove_agent(self, agent):
        self.schedule.remove(agent)
        self.grid.remove_agent(agent)
        
        if type(agent) is LocalAgent:
            self.local_agents.remove(agent)
        elif type(agent) is TouristAgent:
            self.tourist_agents.remove(agent)
    
    def clean_pollution(self):
        """
        Clean pollution
        """
        for patch in self.street_patches.union(self.tourist_spot_patches):
            patch.pollution = max(patch.pollution - Constants.POLLUTION_CLEAN_RATE, 0)
    
    def distribute_town_wealth(self):
        '''
        increase income of locals based on current town wealth
        locals who work in tourism receive a higher proportion
        '''
        params = {True : 0.9, False : 0.1}
        
        for is_pro_tourism, percentage in params.items():
            local_residents = [i for i in self.local_agents if i.is_pro_tourism == is_pro_tourism]
            n_locals = len(local_residents)
            amount_per_person = round(percentage * self.town_wealth // n_locals) if n_locals else 0
        
            for a in local_residents:
                a.income[-1] = a.base_income + amount_per_person
        
        self.town_wealth_hist = self.town_wealth
        self.town_wealth = 0 # reset town wealth
    
    def add_new_tourists(self):
        '''add new tourists'''
        if self.grow_n_tourists:
            """
            mean_tourist_utility_hist = Helper.MODEL_REPORTERS['mean_tourist_utility_hist'](self)
            n_new_tourists = 0

            if mean_tourist_utility_hist >= Constants.TOURIST_UTILITY_HIGH:
                n_new_tourists = 3
            elif mean_tourist_utility_hist >= Constants.TOURIST_UTILITY_LOW:
                n_new_tourists = 2
            else:
                n_new_tourists = 1
            """
            # Have a small probability of introducing a single new tourist regardless
            # of town's satisfaction
            if self.random.random() < 0.1:
                self.n_new_tourists += 1
            
            if self.policy == 'travel_limit':
                self.n_new_tourists = max(min(self.travel_limit - len(self.tourist_agents), self.n_new_tourists), 0)
            
            for i in range(self.n_new_tourists):
                self.add_new_tourist_agent()
    
    def get_mean_factor_for_locals(self, factor, is_pro_tourism):
        agents = [a for a in self.local_agents if a.is_pro_tourism == is_pro_tourism and factor in a.curr_utilities]
        return np.mean([a.curr_utilities[factor] for a in agents]) if agents else 0
    
    def step(self):
        # update tourist satisfaction history
        self.mean_tourist_utility_hist.pop(0)
        self.mean_tourist_utility_hist.append([])
        
        if self.collect_data:
            self.datacollector.collect(self)
        self.schedule.step()
        
        self.clean_pollution()
        self.distribute_town_wealth()
        self.add_new_tourists()
        
        self.debug = ''#str(list(self.local_agents)[0].movement_behaviour)
    
    def run_model(self, n_steps=200):
        for i in range(n_steps):
            self.step()